In [ ]:
import numpy as np
import cupy as cp
import cupyx.scipy.sparse as sp
import cupyx.scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
from scipy.constants import pi, speed_of_light, elementary_charge, electron_mass, hbar
import plotly.graph_objects as go

me_SI = electron_mass
hbar_SI = hbar   
e_SI = elementary_charge
c_SI = speed_of_light

meV = e_SI * 1e-3
nm = 1e-9                    
ps = 1e-12                    

c = c_SI * ps / nm           
hbar_meV_ps = hbar_SI / (meV * ps)  
me = me_SI * c_SI**2 / meV / c**2    

hbar = hbar_meV_ps
m = me
omega = 2 / hbar
vQD = 50

x_min = -75
x_max = 150

x0 = 0
x1 = 75
t0 = 0
t1 = 2

t_min = 0
t_max = t1 + 5

In [ ]:
data = np.load("../results/analytical/crank_2d.npz")

In [ ]:
real = data["real"]
img = data["img"]
x_values = data["x_values"]
y_values = data["y_values"]
t_values = data["t_values"]	

In [ ]:
skip_x = 5
skip_y = 5
skip_t = 5

x_ds = x_values[::skip_x]
y_ds = y_values[::skip_y]
t_ds = t_values[::skip_t]

X, Y, T = np.meshgrid(x_ds, y_ds, t_ds, indexing='ij')

real_ds = real[::skip_x, ::skip_y, ::skip_t]
imag_ds = img[::skip_x, ::skip_y, ::skip_t]
psi_sq_ds = real_ds**2 + imag_ds**2

fig = go.Figure(data=go.Volume(
	x=X.flatten(),
    y=Y.flatten(),
    z=T.flatten(),
    value=psi_sq_ds.flatten(),
    isomin=0,
    isomax=np.max(psi_sq_ds),
    opacity=0.1,
    surface_count=25,
    colorscale='Viridis',
))

fig.update_layout(
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='t',
    ),
    title='3D Volume Plot of |ψ(x, y, t)|²',
    margin=dict(l=0, r=0, b=0, t=50),
    width=1024,
    height=1024
)

fig.show()

In [ ]:
from scipy.integrate import simpson

norms = []

for i in range(1000):
	psi_sq_i = real[:, :, i] ** 2 + img[:, :, i] ** 2
	norms_x = []

	for j in range(1000):
		norm_x = simpson(psi_sq_i[:, j], x=x_values)
		norms_x.append(norm_x)

	normalization = simpson(norms_x, x=y_values)
	norms.append(np.round(normalization, 5))
  
fig, ax = plt.subplots(figsize=(8, 5), dpi=200)

ax.plot(t_values, norms, '-', label='Normalization')
ax.axhline(1.0, color='red', linestyle='--', label='Expected = 1')

ax.yaxis.get_major_formatter().set_useOffset(False)

ax.set_xlabel('Time (ps)')
ax.set_ylabel('Normalization')
ax.set_title('Wavefunction Normalization Over Time')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# from scipy import special

# psi_pinn = real + 1j * img

# def quantum_center(t):
#     if t < t1:
#         return x0
#     elif t < t1 + (x1 - x0) / vQD:
#         return x0 + vQD * (t - t1)
#     else:
#         return x1
    
# def psi_x_n(x, t, n):
#     A = 1 / np.sqrt((2 ** n) * special.factorial(n))
#     B = ((m * omega) / (np.pi * hbar)) ** (1/4)
#     C = np.exp((-m * omega * (x - quantum_center(t)) ** 2) / (2 * hbar))
#     D = special.eval_hermite(n, np.sqrt((m * omega) / hbar) * (x - quantum_center(t)))
#     return A * B * C * D

# def psi_x_t_n(x, t, n):
#     A = psi_x_n(x, t, n)
#     En = (n + (1/2)) * hbar * omega
#     B = np.exp(-(1j * En * t) / hbar)
#     return A * B

# lambda0, lambda1 = [], []
# lambda2, lambda3 = [], []
# lambda4, lambda5 = [], []

# for i, t in enumerate(t_values):
#     psi = psi_pinn[:, i]
    
#     psi0_t = psi_x_t_n(x_values, t, 0)
#     psi1_t = psi_x_t_n(x_values, t, 1)
#     psi2_t = psi_x_t_n(x_values, t, 2)
#     psi3_t = psi_x_t_n(x_values, t, 3)
#     psi4_t = psi_x_t_n(x_values, t, 4)
#     psi5_t = psi_x_t_n(x_values, t, 5)
    
#     lambda0_i = simpson(psi * np.conj(psi0_t), x=x_values)
#     lambda1_i = simpson(psi * np.conj(psi1_t), x=x_values)
#     lambda2_i = simpson(psi * np.conj(psi2_t), x=x_values)
#     lambda3_i = simpson(psi * np.conj(psi3_t), x=x_values)
#     lambda4_i = simpson(psi * np.conj(psi4_t), x=x_values)
#     lambda5_i = simpson(psi * np.conj(psi5_t), x=x_values)
    
#     lambda0.append(np.abs(lambda0_i) ** 2)
#     lambda1.append(np.abs(lambda1_i) ** 2)
#     lambda2.append(np.abs(lambda2_i) ** 2)
#     lambda3.append(np.abs(lambda3_i) ** 2)
#     lambda4.append(np.abs(lambda4_i) ** 2)
#     lambda5.append(np.abs(lambda5_i) ** 2)

# plt.figure(figsize=(8,5), dpi=200)
# plt.plot(t_values, lambda0, label='|lambda0(t)| (ground state)')
# plt.plot(t_values, lambda1, label='|lambda1(t)| (first excited)')
# plt.plot(t_values, lambda2, label='|lambda2(t)| (second excited)')
# plt.plot(t_values, lambda3, label='|lambda3(t)| (third excited)')
# plt.plot(t_values, lambda4, label='|lambda4(t)| (forth excited)')
# plt.plot(t_values, lambda5, label='|lambda5(t)| (fifth excited)')
# plt.xlabel("Time (ps)")
# plt.ylabel("Probability weight")
# plt.title("Time-dependent weights of ground and first excited states")
# plt.legend()
# plt.grid(True)
# plt.show()


In [ ]:
# np.savez("../results/analytical/crank_movement_vQD20_weights.npz", 
#         lambda0_analytical=lambda0,
#         lambda1_analytical=lambda1,
#         lambda2_analytical=lambda2,
#         lambda3_analytical=lambda3,
#         lambda4_analytical=lambda4,
#         lambda5_analytical=lambda5,
#         psi_sq_analytical=psi_sq
# 	)

In [ ]:
arr = np.arange(2.1, 7, .25)
print(len(arr))
print(arr)

In [ ]:
d3 = np.zeros((5, 5, 5))

In [ ]:
d3[1, 1, 1] = 111
d3[1, 0, 1] = 101
d3[1, 1, 0] = 110

In [ ]:
d3

In [ ]:
d3[1, :, 1]

In [ ]:
d3[1, 1, :]

In [ ]:
d3[:, :, 1]